In [1]:
import json

In [2]:
data = './data/outbound-requests-by-domain-2.json'

with open(data, 'rb') as f:
    data = json.load(f)

In [7]:
article = list(data.keys())[0]

In [17]:
data[article].keys()

dict_keys(['content', 'date', 'title', 'outboundRequests'])

In [13]:
data[article]['outboundRequests']

['g1.nyt.com',
 'www.nytimes.com',
 'static01.nyt.com',
 'static.chartbeat.com',
 'www.googletagmanager.com',
 'a.et.nytimes.com',
 'samizdat-graphql.nytimes.com',
 'als-svc.nytimes.com',
 'rumcdn.geoedge.be',
 'c.amazon-adsystem.com',
 'myaccount.nytimes.com',
 'securepubads.g.doubleclick.net',
 'mab.chartbeat.com',
 'ib.adnxs.com',
 'prebid.media.net',
 'tlx.3lift.com',
 'nytimes-d.openx.net',
 'fastlane.rubiconproject.com',
 'sb.scorecardresearch.com',
 'dd.nytimes.com',
 'meter-svc.nytimes.com',
 'a.nytimes.com',
 'edge.microsoft.com',
 'purr.nytimes.com',
 'adservice.google.com',
 'www.google-analytics.com',
 'tags.bluekai.com',
 '5290727.fls.doubleclick.net',
 'tags.bkrtx.com',
 'a1.nyt.com',
 'cdn.brandmetrics.com',
 'insight.adsrvr.org',
 'pagead2.googlesyndication.com',
 'e327286fd7334e174b6dd0a51dd35995.safeframe.googlesyndication.com',
 'mwcm.nytimes.com',
 'tpc.googlesyndication.com',
 'pnytimes.chartbeat.net',
 'stags.bluekai.com',
 'platform.iteratehq.com',
 's.amazon-ads